In [74]:
import pandas as pd
data_df = pd.read_excel('hawaii.xlsx')
data_df.head()
data_df.shape

(877, 9)

In [75]:
# filter duplicates within the file
def remove_duplicates(data_df):
    columns_to_check = ['Company', 'Number', 'Address']
    data_df_without_duplicates = data_df[~data_df.duplicated(subset=columns_to_check, keep='first')]
    return data_df_without_duplicates

unique_data_df = remove_duplicates(data_df)
unique_data_df.shape

(877, 9)

In [76]:
# filter the rows that exist in the master g-maps file
def remove_duplicates_with_master_file(g_maps_master_df, new_file_df):
    columns_to_check = ['Company', 'Number', 'Address']
    original_columns = new_file_df.columns
    merged_df = pd.merge(new_file_df, g_maps_master_df, on=columns_to_check, how='left', indicator=True)
    unique_rows_df = merged_df[merged_df['_merge'] == 'left_only']
    y_columns = [col for col in unique_rows_df.columns if col.endswith('_y')]
    unique_rows_df = unique_rows_df.drop(columns=y_columns + ['_merge'])   
    unique_rows_df.columns = original_columns
    return unique_rows_df

g_maps_master_df = pd.read_excel('g_maps_master_file.xlsx')
unique_from_master_df = remove_duplicates_with_master_file(g_maps_master_df, unique_data_df)
unique_from_master_df.head()
unique_from_master_df.shape

(877, 9)

In [77]:
# save the unique data to the g-maps master file
unique_from_master_df.to_csv('add_to_master_map.csv', index=False)

# save the data without domains to a file for manual review
unique_from_master_df[unique_from_master_df['Domain'].isnull()].to_csv('add_to_manual_review.csv', index=False)

## Data after this point needs to be further processed with automation with email finder, vendors finder etc. The data without domains needs to be added to the file called manual

In [78]:
# Filter based on existing domains
def remove_duplicates(existing_domains, new_file_df):
    existing_domains = existing_domains.rename(columns={'Web_Domain': 'Domain'})
    common_domains = set(existing_domains['Domain']).intersection(set(new_file_df['Domain']))
    filtered_new_df = new_file_df[~new_file_df['Domain'].isin(common_domains)]
    return filtered_new_df

existing_domains_df = pd.read_csv('New_Leads_Domains_Look-Up_Query_De-Duplication.csv')
filtered_based_on_existing_domains = remove_duplicates(existing_domains_df, unique_from_master_df)
filtered_based_on_existing_domains.head()
filtered_based_on_existing_domains.shape


(552, 9)

In [79]:
# Filter based on business types
business_types_df = pd.read_csv('business_types_supported.csv')
unwanted_types = business_types_df[business_types_df['Include (Y/N)'] == 'No']['Google Category'].tolist()
filtered_df = filtered_based_on_existing_domains[~filtered_based_on_existing_domains['Business Type'].isin(unwanted_types)]
display(filtered_df.head())
filtered_df.shape

,Company,Number,Address,Domain,Website,Business Type,Business Status,State,Keyword
0,24-VII Danceforce,+1 808-913-4904,"46-056 Kamehameha Hwy, Kaneohe, HI 96744, Unit...",247dance.org,http://247dance.org/,Dance school,NaN,HI,Pole Dance Studio
1,24 Hour Fitness,+1 808-545-5510,"1000 Bishop St, Honolulu, HI 96813, United States",24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio
2,24 Hour Fitness,+1 808-951-7677,"1680 Kapiolani Blvd, Honolulu, HI 96814, Unite...",24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio
3,24 Hour Fitness,+1 808-396-2424,"7120 Kalanianaʻole Hwy, Honolulu, HI 96825, Un...",24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio
4,24 Hour Fitness,+1 808-377-6855,"91-5431 Kapolei Pkwy #1700, Kapolei, HI 96707,...",24go.co,https://www.24hourfitness.com/gyms/kapolei-hi/...,Gym,NaN,HI,Fitness Studio


(439, 9)

In [ ]:
# Add composite key using domain and address
filtered_df['Lead_id'] = filtered_df.apply(lambda x: f"{x['Domain']}_{x['Address']}".replace(' ', '_'), axis=1)
filtered_df.head()


C:\Users\AsharKhan\AppData\Local\Temp\ipykernel_13092\2889822839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['lead_id'] = filtered_df.apply(lambda x: f"{x['Domain']}_{x['Address']}".replace(' ', '_'), axis=1)


,Company,Number,Address,Domain,Website,Business Type,Business Status,State,Keyword,lead_id
0,24-VII Danceforce,+1 808-913-4904,"46-056 Kamehameha Hwy, Kaneohe, HI 96744, Unit...",247dance.org,http://247dance.org/,Dance school,NaN,HI,Pole Dance Studio,"247dance.org_46-056_Kamehameha_Hwy,_Kaneohe,_H..."
1,24 Hour Fitness,+1 808-545-5510,"1000 Bishop St, Honolulu, HI 96813, United States",24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio,"24go.co_1000_Bishop_St,_Honolulu,_HI_96813,_Un..."
2,24 Hour Fitness,+1 808-951-7677,"1680 Kapiolani Blvd, Honolulu, HI 96814, Unite...",24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio,"24go.co_1680_Kapiolani_Blvd,_Honolulu,_HI_9681..."
3,24 Hour Fitness,+1 808-396-2424,"7120 Kalanianaʻole Hwy, Honolulu, HI 96825, Un...",24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio,"24go.co_7120_Kalanianaʻole_Hwy,_Honolulu,_HI_9..."
4,24 Hour Fitness,+1 808-377-6855,"91-5431 Kapolei Pkwy #1700, Kapolei, HI 96707,...",24go.co,https://www.24hourfitness.com/gyms/kapolei-hi/...,Gym,NaN,HI,Fitness Studio,"24go.co_91-5431_Kapolei_Pkwy_#1700,_Kapolei,_H..."


In [81]:
# Add scheme to the domain
filtered_df['Domain'] = filtered_df['Domain'].apply(lambda x: f"http://{x}")
filtered_df.head()

C:\Users\AsharKhan\AppData\Local\Temp\ipykernel_13092\3920613413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Domain'] = filtered_df['Domain'].apply(lambda x: f"http://{x}")


,Company,Number,Address,Domain,Website,Business Type,Business Status,State,Keyword,lead_id
0,24-VII Danceforce,+1 808-913-4904,"46-056 Kamehameha Hwy, Kaneohe, HI 96744, Unit...",http://247dance.org,http://247dance.org/,Dance school,NaN,HI,Pole Dance Studio,"247dance.org_46-056_Kamehameha_Hwy,_Kaneohe,_H..."
1,24 Hour Fitness,+1 808-545-5510,"1000 Bishop St, Honolulu, HI 96813, United States",http://24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio,"24go.co_1000_Bishop_St,_Honolulu,_HI_96813,_Un..."
2,24 Hour Fitness,+1 808-951-7677,"1680 Kapiolani Blvd, Honolulu, HI 96814, Unite...",http://24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio,"24go.co_1680_Kapiolani_Blvd,_Honolulu,_HI_9681..."
3,24 Hour Fitness,+1 808-396-2424,"7120 Kalanianaʻole Hwy, Honolulu, HI 96825, Un...",http://24go.co,https://www.24hourfitness.com/gyms/honolulu-hi...,Gym,NaN,HI,Fitness Studio,"24go.co_7120_Kalanianaʻole_Hwy,_Honolulu,_HI_9..."
4,24 Hour Fitness,+1 808-377-6855,"91-5431 Kapolei Pkwy #1700, Kapolei, HI 96707,...",http://24go.co,https://www.24hourfitness.com/gyms/kapolei-hi/...,Gym,NaN,HI,Fitness Studio,"24go.co_91-5431_Kapolei_Pkwy_#1700,_Kapolei,_H..."


In [82]:
filtered_df.to_csv('filtered_leads.csv', index=False)